### 1. Pivot Tables - pivot_table()

In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

In [ ]:
'''
엑셀... pivot table 기능이 있다. 이것과 동일한 기능
Pandas에서는 pivot_table() 함수를 제공
피봇함수 역할?
DataFrame의 데이타를 ReShape하는 방법중 하나.
구체적인 방법론
1.
2.
3.
'''

In [4]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}

df1 = DataFrame(data)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [13]:
#df1.pivot_table? (value, index, column)
# df1.pivot_table('인구', '도시', '연도')
# df1.pivot_table('인구', columns='연도', index='도시')
# df1.pivot_table('인구', columns=['연도'], index=['도시'])
df1.pivot_table('인구', columns=['연도'], index=['도시'], margins=True)

연도,2005,2010,2015,All
도시,,,,
부산,3512547.0,3393191.0,3448737.0,3.451492e+06
서울,9762546.0,9631482.0,9904312.0,9.766113e+06
인천,NaN,263203.0,2890451.0,1.576827e+06
All,6637546.5,4429292.0,5414500.0,5.350809e+06


In [10]:
df1['인구'].mean()

5350808.625

In [14]:
df1.pivot_table(['인구'], index=['연도', '도시'])
# index는 결국 groupby 동일한 역할을 하는 옵션이다.

인구
연도   도시         
2005 부산  3512547
     서울  9762546
2010 부산  3393191
     서울  9631482
     인천   263203
2015 부산  3448737
     서울  9904312
     인천  2890451

#### 실전데이타 응용하기

In [15]:
tipdf = pd.read_csv('../data/tips.csv')
tipdf.head(1)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2.0


In [20]:
'''
해당 tips 데이타의 분석의 목표는 사실상.. tip 비용
식사대금 대비 팁의 비율이 어떤지 알수 있는 컬럼 하나를 더 추가해보자
tip_pct = Tip / total_bill
'''

tipdf['tip_pct'] = tipdf['tip'] / tipdf['total_bill']
tipdf.tail()

#NaN 누락데이터 제거..원본제거
tipdf.dropna(inplace=True)
tipdf.tail()

# 데이터들에 대한 간략한 통계정보 출력, 분포
tipdf.describe()
tipdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 0 to 243
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    float64
 7   tip_pct     244 non-null    float64
dtypes: float64(4), object(4)
memory usage: 17.2+ KB


In [23]:
#문제 1. tipdf에서 피봇함수를 사용, day컬럼으로 세분화
#rount(n) : 소수점 n+1자리에서 반올림
tipdf.pivot_table(index='day', aggfunc='mean').round(2)

,size,tip,tip_pct,total_bill
day,,,,
Fri,2.11,2.73,0.17,17.15
Sat,2.52,2.99,0.15,20.44
Sun,2.84,3.26,0.17,21.41
Thur,2.45,2.77,0.16,17.68


In [24]:
# 문제 2. tipdf에서 모든 데이터를 성별로 세분화, 흡연연부별로 세분화, 소수점 2자리까지 출력
tipdf.pivot_table(index=['sex', 'smoker']).round(2)

size   tip  tip_pct  total_bill
sex    smoker                                 
Female No      2.59  2.77     0.16       18.11
       Yes     2.24  2.93     0.18       17.98
Male   No      2.71  3.11     0.16       19.79
       Yes     2.50  3.05     0.15       22.28

In [29]:
'''
pivot_table() 함수 인자값 속성의 의미

index - 세분화, 그룹핑 하는 기준
column - 컬럼값
values = 데이터프레임을 채우고자 하는 값


'''


# tipdf.pivot_table {
#     values=['total_bill', 'tip'], # 채우고자 하는 값
#     index = ['sex', 'day'], # 인덱스 정의
#     column = 'smoker' #column 정의
# }

'\npivot_table() 함수 인자값 속성의 의미\n\nindex - 세분화, 그룹핑 하는 기준\ncolumn - 컬럼값\nvalues = 데이터프레임을 채우고자 하는 값\n\n\n'

In [30]:
tipdf.pivot_table('tip', 'sex', 'smoker')

smoker,No,Yes
sex,,
Female,2.773519,2.931515
Male,3.113402,3.051167


In [35]:
# 문제 3
# 성별에 따른 평균 팁의 비율

# 1) 피봇테이블 사용
tipdf.pivot_table('tip_pct', 'sex').round(2)
# 2) groupby()
tipdf.groupby('sex')[['tip_pct']].mean().round(2)

,tip_pct
sex,
Female,0.17
Male,0.16


In [36]:
#문제 4 groupby로 해결

# 각, 그룹에서 가장 많은 팁과 가장 적은 팁의 차이를 디스플레이
# 단, 사용자정의 함수를 지정...max_min_Tip으로 정의

def max_min_Tip(x) :
    return x.max() - x.min()

tipdf.groupby(['sex', 'smoker'])[['tip']].agg(max_min_Tip)
    


tip
sex    smoker      
Female No      4.20
       Yes     5.50
Male   No      7.75
       Yes     9.00